In [13]:
import anthropic
import os
from dotenv import load_dotenv
load_dotenv()
client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)
# print(client.api_key)
message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    temperature=0.0,
    system="translate the input into English and respond in Hebrew",
    messages=[
        {"role": "user", "content": "מה קורה?"}
    ]
)

print(message.content)

[TextBlock(text='שלום! אני שמח לעזור לך. מה אפשר לעזור לך?', type='text')]


In [18]:
def get_response(input):
    message = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.0,
        system="answer in Hebrew to the user question",
        messages=[
            {"role": "user", "content": input}
        ]
    )
    return message.content

In [20]:
print(get_response("ספר לי על המלחמה של ישראל וחמאס בשנת 2023"))

[TextBlock(text='להלן סיכום של המלחמה בין ישראל וחמאס בשנת 2023:\n\n• הסלמה החלה בעקבות עימותים באזור ירושלים והר הבית, שהובילו לעימותים צבאיים בין ישראל לחמאס ברצועת עזה.\n\n• חמאס ירה מאות רקטות לעבר ערים בישראל, בעוד ישראל תקפה מטרות צבאיות ברצועה בתגובה.\n\n• הלחימה נמשכה כשבועיים, תוך הרס נרחב בעזה והרוגים משני הצדדים. ישראל הפעילה את מערכת ההגנה האווירית "כיפת ברזל" להתמודדות עם הרקטות.\n\n• בסופו של דבר, הושג הסכם הפסקת אש בתיווך בינלאומי. הצדדים הסכימו על הפסקת אש, תמורת הקלות על המצור הישראלי על עזה ושיקום התשתיות שנהרסו.\n\n• המלחמה הביאה לנזקים כבדים בעזה ואבדות אנושיות משני הצדדים, אך לא הובילה לפריצת דרך משמעותית בסכסוך הישראלי-פלסטיני.', type='text')]


In [14]:
def translate_into_hebrew(input_text):
    message = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=4000,
        temperature=0.0,
        system="translate the input into Hebrew return the translated text",
        messages=[
            {"role": "user", "content": input_text}
        ]
    )
    return message.content

In [15]:
import PyPDF2
def read_pdf_file(file_path):
    reader=PyPDF2.PdfReader(file_path)
    pages=[]
    for page in reader.pages:
        pages.append(page.extract_text())
    return pages


In [16]:
pages=read_pdf_file("/Users/matansharon/python/Data_science/some tests/ivrit_ai.pdf")

In [17]:
translated_text=translate_into_hebrew(pages[2])

In [ ]:
print(translated_text[0].text)